This notebook will try to implement a vgg 16 model pretrained on imagenet provided in keras

vgg16 usage:
keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [58]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Conv2D

from keras.layers import MaxPool2D
from keras.layers import Flatten

from keras.optimizers import Adam

from keras.layers import Dropout

from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model

from keras.applications.vgg16 import decode_predictions

In [32]:
train_img_file = 'train_img.npy'
train_label_file = 'train_label.npy'

test_img_file = 'test_img.npy'
test_label_file = 'test_label.npy'

train_img = np.load(train_img_file)
train_label = np.load(train_label_file)

test_img = np.load(test_img_file)
test_label = np.load(test_label_file)

#we need to move the number of images axis to the first one
train_img = np.moveaxis(train_img, -1, 0)
test_img = np.moveaxis(test_img, -1, 0)

print(train_img.shape)
print(test_img.shape)

(98, 224, 224, 3)
(98, 224, 224, 3)


In [52]:
#we will first try a complete vgg16 model for prediction
vgg16_model_complete = VGG16(include_top=True, weights='imagenet', 
                    input_tensor=None, input_shape=(224, 224, 3), pooling=None)

print(vgg16_model_complete.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [61]:
vgg16_model_complete.compile(loss='categorical_crossentropy', 
                 optimizer='adam',
                metrics=['accuracy'])

yhat = vgg16_model_complete.predict(train_img)

# convert the probabilities to class labels
label = decode_predictions(yhat)

#convert to a numpy array
label = np.array(label)

label

predicted_label = label[:, 0, 1]

print(predicted_label)

["jack-o'-lantern" 'bucket' 'candle' 'tray' 'tray' 'jellyfish' 'bubble'
 'bubble' 'bubble' 'Petri_dish' 'Petri_dish' 'Petri_dish' 'Petri_dish'
 'Petri_dish' 'Petri_dish' 'bubble' 'bubble' 'pick' 'pick' 'whistle'
 'tennis_ball' 'whistle' 'nematode' 'nematode' 'bubble' 'nematode'
 'traffic_light' 'traffic_light' 'traffic_light' 'jellyfish' 'vase' 'puck'
 'bubble' 'hair_slide' 'puck' 'bucket' 'bucket' 'bucket' 'bubble' 'bubble'
 'bubble' 'bubble' 'bubble' 'bubble' 'puck' 'bubble' 'bubble' 'puck'
 'puck' 'bubble' 'bubble' 'puck' 'puck' 'puck' 'bucket' 'puck' 'puck'
 'puck' 'football_helmet' 'puck' 'iPod' 'CD_player' 'traffic_light'
 'CD_player' 'digital_clock' 'digital_clock' 'tray' 'tray' 'tray' 'tray'
 'tray' 'tray' 'tray' 'jersey' 'bubble' 'bubble' 'bubble' 'soccer_ball'
 'pick' 'Petri_dish' 'Petri_dish' 'Petri_dish' 'Petri_dish' 'Petri_dish'
 'face_powder' 'Petri_dish' 'Petri_dish' 'Petri_dish' 'face_powder'
 'face_powder' 'Petri_dish' 'bucket' 'bucket' 'tray' 'hair_slide'
 'traffic_li

Now we will try to remove the top 3 fully connected layers and try to fine tune it for our own classification purposes

In [60]:
#add our own fully connected layers for the final classification

# create the base pre-trained model
base_model = VGG16(include_top=False, weights='imagenet', 
                    input_tensor=None, input_shape=(224, 224, 3), pooling=None)

x = base_model.output
#flattentit
x = Flatten()(x)
# let's add a fully-connected layer
x = Dense(4096, activation='relu')(x)
#another fully-connected layer
x = Dense(4096, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, kernel_initializer='normal', activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
#now we can start to fine-tune the model
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

